In [2]:
%load_ext autoreload
%autoreload 1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.decomposition import KernelPCA

In [38]:
# df_rain = pd.read_csv('datasets/benchmark/weatherAUS_clean_train.csv')
df_rain = pd.read_csv('datasets/normalized_benchmark/weatherAUS_clean_train.csv')
rain_feats = df_rain.columns.values
rain_x = df_rain[rain_feats[:-1]]
rain_y = df_rain[rain_feats[-1]]
rain_x_tr, rain_x_te, rain_y_tr, rain_y_te = train_test_split(rain_x, rain_y, test_size=0.2)

In [39]:
rain_x.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindGustDir_E,WindGustDir_W,WindDir9am_N,WindDir9am_S,WindDir9am_E,WindDir9am_W,WindDir3pm_N,WindDir3pm_S,WindDir3pm_E,WindDir3pm_W
0,-0.255460,-0.745831,1.853462,1.120662,-0.448305,1.881700,0.511066,1.263930,1.315774,1.344504,...,-0.722969,0.674722,2.060423,-0.744279,-0.745346,-0.766083,-0.760850,-0.795909,-0.732678,2.189419
1,-0.805911,-0.984383,0.379110,1.120662,0.132106,0.362728,0.511066,0.033671,0.675162,0.076216,...,-0.722969,0.674722,0.617992,-0.744279,-0.745346,0.801350,1.315759,-0.795909,-0.732678,0.179914
2,-0.695821,-1.419388,0.084240,-1.027741,-0.448305,2.572141,3.304530,2.382348,0.995468,0.347992,...,-0.722969,2.173966,-0.824440,-0.744279,-0.745346,2.368782,-0.760850,-0.795909,-0.732678,2.189419
3,0.405081,1.106451,-0.332048,-0.670768,0.132106,0.017507,0.743854,0.481038,-1.033137,-0.920296,...,-0.722969,-0.824521,2.060423,-0.744279,-0.745346,-0.766083,1.315759,-0.795909,-0.732678,0.179914
4,-1.041819,-0.226631,-0.332048,0.201833,0.132106,-1.156244,0.161883,-0.301854,-0.285756,-0.150264,...,-0.722969,0.674722,0.137181,-0.744279,-0.745346,1.323827,-0.760850,0.239841,1.429113,-0.824839


In [40]:
rain_y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: RainTomorrow, dtype: float64

In [6]:
import subset_pca
%aimport subset_pca

In [41]:
spca = subset_pca.SubsetPCA(type='dim', value=10)

In [42]:
spca.fit(rain_x_tr)

([{'Cloud3pm', 'Cloud9am', 'Sunshine'},
  {'WindDir3pm_E', 'WindDir3pm_W', 'WindGustDir_E', 'WindGustDir_W'},
  {'Evaporation'},
  {'Humidity3pm', 'Humidity9am', 'MaxTemp', 'MinTemp', 'Temp3pm', 'Temp9am'},
  {'WindDir3pm_N',
   'WindDir3pm_S',
   'WindDir9am_N',
   'WindDir9am_S',
   'WindGustDir_N',
   'WindGustDir_S'},
  {'Rainfall'},
  {'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'},
  {'RainToday'},
  {'WindDir9am_E', 'WindDir9am_W'},
  {'Pressure3pm', 'Pressure9am'}],
 [PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1)])

In [43]:
small_x_tr = spca.transform(rain_x_tr)
small_x_te = spca.transform(rain_x_te)

In [44]:
from sklearn.linear_model import LogisticRegression as lr
from sklearn.metrics import confusion_matrix as cm

In [45]:
clf = lr(penalty='l2').fit(small_x_tr, rain_y_tr)
print(clf.score(small_x_tr, rain_y_tr), clf.score(small_x_te, rain_y_te))
tr_cm = cm(clf.predict(small_x_tr), rain_y_tr)
te_cm = cm(clf.predict(small_x_te), rain_y_te)
print(tr_cm[0] / sum(tr_cm[0]), tr_cm[1] / sum(tr_cm[1]))
print(te_cm[0] / sum(te_cm[0]), te_cm[1] / sum(te_cm[1]))
# print(cm(clf.predict(small_x_tr), rain_y_tr), '\n', cm(clf.predict(small_x_te), rain_y_te))

print('Coefficients:', clf.coef_)

0.7384375 0.7478125
[0.73145356 0.26854644] [0.25405318 0.74594682]
[0.73410055 0.26589945] [0.23757263 0.76242737]
Coefficients: [[-0.56610491  0.07826936  0.00971408  0.19956903 -0.11066422  0.14963912
   0.17856359  0.38401395  0.00651896  0.39533739]]


In [46]:
clf_big = lr(penalty='l2').fit(rain_x_tr, rain_y_tr)
print(clf_big.score(rain_x_tr, rain_y_tr), clf_big.score(rain_x_te, rain_y_te))
tr_cm = cm(clf_big.predict(rain_x_tr), rain_y_tr)
te_cm = cm(clf_big.predict(rain_x_te), rain_y_te)
print(tr_cm[0] / sum(tr_cm[0]), tr_cm[1] / sum(tr_cm[1]))
print(te_cm[0] / sum(te_cm[0]), te_cm[1] / sum(te_cm[1]))

0.788359375 0.7953125
[0.78188174 0.21811826] [0.20479023 0.79520977]
[0.79033271 0.20966729] [0.19975109 0.80024891]


In [25]:
spca.visualize_components()